In [61]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25")

In [3]:
# from langchain_groq import ChatGroq
# import os
# llm=ChatGroq(model_name="Gemma2-9b-It")

# Simple AI Assistant

In [14]:
while True:
    question = input("Type Your Question. if you want to quit type 'exit'")
    if question != "exit":
        print(llm.invoke(question).content)
    else:
        print("Goodbye !!!")
        break

Hi Ravi! It's nice to meet you.

I'm a large language model, trained by Google.

How can I help you today?
I do not have access to your personal information, including your name. I am an AI and don't have the ability to know who you are unless you tell me.
Goodbye !!!


In [15]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import  AIMessage

In [16]:
store={}

In [17]:
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [18]:
config = {"configurable":{"session_id":"firstchat"}}

In [19]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [20]:
model_with_memory.invoke(("hii I'm Ravi Avaiya"),config=config).content

"Hi Ravi Avaiya! It's nice to meet you.\n\nHow can I help you today?"

In [21]:
model_with_memory.invoke(("Now Tell me What is my name?"),config=config).content

'You told me your name is Ravi Avaiya in your first message.'

In [25]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="hii I'm Ravi Avaiya", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Ravi Avaiya! It's nice to meet you.\n\nHow can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro-exp-03-25', 'safety_ratings': []}, id='run-41a93623-a72b-49be-bfcf-39c5f27a06ac-0', usage_metadata={'input_tokens': 8, 'output_tokens': 366, 'total_tokens': 374, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Now Tell me What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='You told me your name is Ravi Avaiya in your first message.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro-exp-03-25', 'safety_ratings': []}, id='run-d238f82d-94f3-4954-9825-22b

# Rag WIth LCEL

In [36]:
from langchain_community.document_loaders import  TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [37]:
# Reading Text From File
loader = TextLoader("D:\LangGraph-End-To-End\data\llama3.txt")
docs= loader.load()

# Splitting Text into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(docs)
doc_strings = [doc.page_content for doc in new_docs]

# Embedding Text Chunks
db = FAISS.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 1})

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\ravia\AppData\Local\Temp\ipykernel_14828\2660421344.py:2: SyntaxWarning: invalid escape sequence '\L'
  loader = TextLoader("D:\LangGraph-End-To-End\data\llama3.txt")


In [39]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [40]:
retrival_chain = (
    RunnableParallel(
    {"context":retriever,"question":RunnablePassthrough()})
    | prompt
    | llm   
    | StrOutputParser()
)

In [41]:
question = "what is llama?  can you highlight 3 important points?"
print(retrival_chain.invoke(question))

Based on the provided context:

The context does not explicitly define what Llama is. However, it does provide the following information, which can be considered important points:

1.  Llama exists in different versions.
2.  Versions of Llama were made accessible outside.

The context does not provide enough information for a third distinct point or a definition of what Llama is.


# Let's Start with Tools and Agents

In [42]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper



In [43]:
api_wrapper = WikipediaAPIWrapper()
tool= WikipediaQueryRun(api_wrapper=api_wrapper)

In [44]:
tool.name

'wikipedia'

In [45]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [46]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [47]:
print(tool.run({"query":"Langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information. Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves larg

## Youtube Search

In [53]:
from langchain_community.tools import YouTubeSearchTool


In [55]:
youtube=YouTubeSearchTool()
youtube.name

'youtube_search'

In [58]:
youtube.run("Ravi Avaiya")

"['https://www.youtube.com/shorts/r3eYj9B4Klk', 'https://www.youtube.com/watch?v=4Nx8kEWIVsU&pp=ygULUmF2aSBBdmFpeWE%3D']"

## Google Search

In [59]:
from langchain_community.tools import TavilySearchResults

In [62]:
tavily=TavilySearchResults()

In [71]:
search= tavily.invoke({"query":"Which MAtch Today IN IPL 2025?"})
search


[{'title': 'IPL Match Schedule 2025 - Times of India',
  'url': 'https://timesofindia.indiatimes.com/sports/cricket/ipl/schedule',
  'content': 'IPL Schedule 2025: Today IPL Match, Dates, Time, Venue, Teams and Match Time Table Schedule Schedule IPL Schedule 2025: The highly anticipated 2025 edition of the Indian Premier League (IPL) will take place from March 14 to May 25, the Board of Control for Cricket in India (BCCI) has officially informed the franchises. IPL 2025 Schedule The IPL 2025 will begin with an explosive opening match on March 14, while the highly anticipated final is scheduled for May 25, offering fans a spectacular culmination to the season. To stay updated on the IPL 2025 schedule, fixtures, and all breaking news, follow TOI Sports on the website, YouTube channel, and across all major social media platforms.',
  'score': 0.86024034},
 {'title': 'IPL 2025 - Cricket Schedule & Results - ESPNcricinfo',
  'url': 'https://www.espncricinfo.com/series/ipl-2025-1449924/match

In [90]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent, load_tools

In [91]:
tool = load_tools(["wikipedia"],llm)

In [92]:
agent=initialize_agent(tools=tool, llm=llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [93]:
agent.run("What is Current GDP of India?")



> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 25
}
]